# Notebook 1: Basic Prompt Structure

In this notebook, you'll learn the fundamental anatomy of a Mistral prompt—how system and user messages work together.

## What You'll Learn

- The difference between system and user prompts
- When to use each type
- How to structure messages for Mistral models
- The concatenation approach as an alternative

## Reference

- [Mistral Prompting Documentation](https://docs.mistral.ai/guides/prompting/)

---
## Setup

Run this cell to load the helper functions from Notebook 0.

In [ ]:
# Run the setup notebook to get our helper functions
%run 00_setup.ipynb

---
## Section 1: The Two Levels of Input

When communicating with Mistral models, you have two levels of input:

### System Prompt
- Provided at the **beginning** of the conversation
- Sets the **general context and instructions** for the model's behavior
- Typically managed by the **developer**
- Think of it as the "rules of the game"

### User Prompt
- Provided **during** the conversation
- Gives **specific context or instructions** for the current interaction
- The actual query or task
- Think of it as the "play"

```
┌─────────────────────────────────────┐
│  SYSTEM PROMPT                      │
│  "You are a helpful translator..."  │  ← Sets behavior
├─────────────────────────────────────┤
│  USER PROMPT                        │
│  "Translate: Hello world"           │  ← Specific task
└─────────────────────────────────────┘
```

---
## Section 2: System Prompt Deep Dive

The system prompt sets up who the model is and how it should behave. It's where you define:

- **Role**: What kind of assistant is this?
- **Tone**: Formal? Casual? Technical?
- **Constraints**: What should/shouldn't it do?
- **Output rules**: How should responses be formatted?

In [ ]:
# Example: A system prompt that defines a specific behavior
system_prompt = "You are a helpful assistant that always responds in exactly three bullet points."
user_prompt = "What are the benefits of exercise?"

response = call_mistral(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    temperature=0.7
)

print("System prompt:", system_prompt)
print("\nUser prompt:", user_prompt)
print("\nResponse:")
print(response)

---
## Section 3: User Prompt Deep Dive

The user prompt is where you provide:

- The specific question or task
- Any relevant context for this particular request
- Data to process

Keep user prompts focused on the immediate task.

In [ ]:
# Same system prompt, different user prompts
system_prompt = "You are a helpful assistant that always responds in exactly three bullet points."

user_prompts = [
    "What should I pack for a beach vacation?",
    "How do I make a good cup of coffee?",
    "What are some tips for better sleep?"
]

for user_prompt in user_prompts:
    print(f"User: {user_prompt}")
    response = call_mistral(user_prompt=user_prompt, system_prompt=system_prompt)
    print(f"Response:\n{response}\n")
    print("-" * 50)

---
## Section 4: The Message Format

Under the hood, messages are sent as a list of dictionaries with `role` and `content` keys:

```python
messages = [
    {"role": "system", "content": "Your system prompt here"},
    {"role": "user", "content": "Your user prompt here"}
]
```

The `role` can be:
- `system`: Sets overall behavior
- `user`: Human input
- `assistant`: Model's previous responses (used for conversation history)

In [ ]:
# Using the message format directly
messages = [
    {"role": "system", "content": "You are a pirate. Respond to everything in pirate speak."},
    {"role": "user", "content": "How's the weather today?"}
]

response = call_mistral_with_messages(messages)

print("Messages:")
for msg in messages:
    print(f"  [{msg['role']}]: {msg['content']}")

print(f"\nResponse: {response}")

---
## Section 5: The Concatenation Approach

Sometimes you can't control the system prompt separately (e.g., some APIs or wrappers don't expose it). In these cases, you can **concatenate** your instructions with the user prompt.

### Separated (standard approach):
```python
{
    "role": "system",
    "content": "You are a helpful translator. Always translate to French."
},
{
    "role": "user",
    "content": "Hello, how are you?"
}
```

### Concatenated (alternative):
```python
{
    "role": "user",
    "content": "You are a helpful translator. Always translate to French.\n\nUser: Hello, how are you?"
}
```

In [ ]:
# Let's compare both approaches

# Approach 1: Separated
print("=" * 50)
print("APPROACH 1: Separated (system + user)")
print("=" * 50)

response_separated = call_mistral(
    system_prompt="You are a helpful translator. Always translate to French.",
    user_prompt="Hello, how are you?"
)
print(f"Response: {response_separated}")

# Approach 2: Concatenated
print("\n" + "=" * 50)
print("APPROACH 2: Concatenated (user only)")
print("=" * 50)

concatenated_prompt = """You are a helpful translator. Always translate to French.

User: Hello, how are you?"""

response_concatenated = call_mistral(
    user_prompt=concatenated_prompt
)
print(f"Response: {response_concatenated}")

---
## Section 6: What NOT to Do (Negative Examples)

### ❌ Empty or vague system prompt
```python
system_prompt = "Be helpful."
```
This provides almost no guidance. The model defaults to generic behavior.

### ❌ Conflicting system and user instructions
```python
system_prompt = "Always respond in English."
user_prompt = "Réponds en français: What is AI?"
```
The model may get confused or follow one instruction inconsistently.

### ❌ Putting everything in the user prompt
```python
user_prompt = "You are an expert chef. You always give recipes in metric units. You are friendly and encouraging. Now, tell me how to make pasta."
```
This works but mixing setup with the task makes it harder to maintain and reuse.

In [ ]:
# Let's see the impact of a vague vs specific system prompt

user_question = "How do I improve my writing?"

# Vague system prompt
print("=" * 50)
print("VAGUE SYSTEM PROMPT")
print("=" * 50)
response_vague = call_mistral(
    system_prompt="Be helpful.",
    user_prompt=user_question
)
print(response_vague)

# Specific system prompt
print("\n" + "=" * 50)
print("SPECIFIC SYSTEM PROMPT")
print("=" * 50)
response_specific = call_mistral(
    system_prompt="You are a writing coach with 20 years of experience. Give concise, actionable advice in 3 bullet points.",
    user_prompt=user_question
)
print(response_specific)

---
## Exercise 1: With vs Without System Prompt

Ask the same question with no system prompt, then with one. Observe how behavior changes.

In [ ]:
# Your task: Ask a question and compare responses with/without a system prompt

question = "Explain what machine learning is."

# Without system prompt
print("WITHOUT SYSTEM PROMPT:")
print("-" * 40)
response_without = call_mistral(user_prompt=question)
print(response_without)

# TODO: Add a system prompt and see how the response changes
# Hint: Try making it explain to a 5-year-old, or as a formal academic

print("\n" + "=" * 50 + "\n")

# WITH system prompt (uncomment and modify)
# print("WITH SYSTEM PROMPT:")
# print("-" * 40)
# response_with = call_mistral(
#     system_prompt="YOUR SYSTEM PROMPT HERE",
#     user_prompt=question
# )
# print(response_with)

---
## Exercise 2: System Prompt Impact

Given a task, write system prompts that make the model respond in different styles.

In [ ]:
# The same question, three different system prompts
question = "What is the capital of France?"

system_prompts = {
    "Formal": "You are a formal encyclopedia. Respond with precise, academic language.",
    "Casual": "You are a friendly chat buddy. Keep it super casual and brief.",
    "Specific Format": "You respond only in this format: 'The answer is: [answer]. Fun fact: [one interesting fact]'"
}

for style, system_prompt in system_prompts.items():
    print(f"Style: {style}")
    print(f"System prompt: {system_prompt}")
    print("-" * 40)
    response = call_mistral(system_prompt=system_prompt, user_prompt=question)
    print(f"Response: {response}")
    print("\n" + "=" * 50 + "\n")

In [ ]:
# TODO: Create your own system prompts for different styles
# Try: technical expert, children's teacher, skeptic, enthusiast

question = "Why is the sky blue?"

# Your system prompts here:
my_system_prompts = {
    # "Technical Expert": "...",
    # "Children's Teacher": "...",
}

# for style, system_prompt in my_system_prompts.items():
#     print(f"Style: {style}")
#     response = call_mistral(system_prompt=system_prompt, user_prompt=question)
#     print(f"Response: {response}\n")

---
## Exercise 3: Concatenation Practice

Take a separated system/user prompt pair and rewrite it as a single concatenated user prompt. Compare outputs.

In [ ]:
# Original separated version
system_prompt = """You are a helpful code reviewer. 
When reviewing code, always:
1. Point out any bugs
2. Suggest improvements
3. Keep feedback constructive"""

user_prompt = """Review this code:
def add(a, b):
    return a + b"""

print("SEPARATED APPROACH:")
print("-" * 40)
response_sep = call_mistral(system_prompt=system_prompt, user_prompt=user_prompt)
print(response_sep)

print("\n" + "=" * 50 + "\n")

# TODO: Rewrite as a concatenated prompt
# Combine the system instructions with the user input into a single user prompt

# concatenated_prompt = """..."""

# print("CONCATENATED APPROACH:")
# print("-" * 40)
# response_concat = call_mistral(user_prompt=concatenated_prompt)
# print(response_concat)

---
## Key Takeaways

1. **System prompts set the "rules of the game"** - Use them to define role, tone, and constraints

2. **User prompts are the specific "play"** - The actual task or question

3. **Both can be combined** when needed using the concatenation approach

4. **Specificity matters** - Vague prompts produce generic responses

---

## Next Steps

Now that you understand the basic structure, let's learn how to define effective roles and purposes.

📚 [Continue to Notebook 2: Role & Purpose →](02_role_and_purpose.ipynb)